In [26]:
# Bloc 3.16 – M2 PR/PCA sur toutes les fenêtres S0

import numpy as np
import pandas as pd
from pathlib import Path

# 3.16.1 – Vérifier df_ts (S0) et df_windows

try:
    df_ts
except NameError:
    raise RuntimeError(
        "df_ts n'est pas défini. Assurez-vous d'avoir exécuté le Bloc 3.0 avant ce bloc."
    )

value_col = "Monthly Mean Total Sunspot Number"

try:
    df_windows
except NameError:
    WINDOW_DEFS_PATH = PHASE2_ROOT / "data_phase2" / "windows" / "window_definitions.csv"
    if not WINDOW_DEFS_PATH.exists():
        raise FileNotFoundError(
            f"window_definitions.csv introuvable à : {WINDOW_DEFS_PATH}.\n"
            "Assurez-vous d'avoir exécuté le Bloc 2.4 / 3.0."
        )
    df_windows = pd.read_csv(
        WINDOW_DEFS_PATH,
        parse_dates=["start_date", "end_date"],
    )

print(f"Nombre total de fenêtres S0 : {len(df_windows)}")

# 3.16.2 – Vérifier la fonction PR (définie en 3.8)

try:
    pr_dimensions_from_cov_eigvals
except NameError:
    raise RuntimeError(
        "La fonction pr_dimensions_from_cov_eigvals n'est pas définie. "
        "Assurez-vous d'avoir exécuté le Bloc 3.8 (M2 sur S0, calibration) avant ce bloc."
    )

# 3.16.3 – Application de M2 sur toutes les fenêtres par combinaison (W,G)

records_M2_S0_all = []

groups = df_windows.groupby(["window_size_months", "stride_months"], sort=True)

for (W, G), group in groups:
    group = group.sort_values("start_index").reset_index(drop=True)
    n_windows = len(group)

    print(f"\nM2 PR/PCA sur toutes les fenêtres S0 pour (W={W}, G={G}) – n_windows={n_windows}")

    # Construction de X_all (n_windows x W)
    rows = []
    for _, row in group.iterrows():
        start_idx = int(row["start_index"])
        end_idx = int(row["end_index"])

        values = df_ts.iloc[start_idx : end_idx + 1][value_col].to_numpy(dtype=float)
        if len(values) != W:
            raise RuntimeError(
                f"Longueur inattendue pour window_id={row['window_id']} "
                f"(attendu W={W}, obtenu {len(values)})."
            )
        rows.append(values)

    X_all = np.vstack(rows)

    # Centrage des colonnes
    X_centered = X_all - X_all.mean(axis=0, keepdims=True)

    # Covariance sur les colonnes
    cov = np.cov(X_centered, rowvar=False)

    # Valeurs propres
    eigvals = np.linalg.eigvalsh(cov)
    eigvals = np.flip(np.sort(eigvals))

    d_PR, d_by_thr = pr_dimensions_from_cov_eigvals(eigvals, var_thresholds=(0.8, 0.9))
    d_80 = d_by_thr[0.8]
    d_90 = d_by_thr[0.9]

    record = {
        "series": "S0",
        "window_size_months": int(W),
        "stride_months": int(G),
        "n_windows": int(n_windows),
        "W": int(W),
        "d_PR": float(d_PR),
        "d_PR_80": int(d_80),
        "d_PR_90": int(d_90),
        "n_eigvals_pos": int((eigvals > 0).sum()),
    }
    records_M2_S0_all.append(record)

    print(
        f"  -> (W={W}, G={G}) : d_PR={d_PR:.3f}, d_PR_80={d_80}, d_PR_90={d_90}"
    )

# 3.16.4 – DataFrame et sauvegarde

df_M2_S0_all = pd.DataFrame.from_records(records_M2_S0_all)

print("\nRésumé M2 (S0, toutes fenêtres) :")
display(df_M2_S0_all)

DEST_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates"
DEST_DIR.mkdir(parents=True, exist_ok=True)

M2_S0_ALL_SUMMARY_PATH = DEST_DIR / "M2_S0_PR_all_windows_summary.csv"
df_M2_S0_all.to_csv(M2_S0_ALL_SUMMARY_PATH, index=False)

print("\nFichier de résultats M2 (S0, PR/PCA, toutes fenêtres) sauvegardé :")
print(f"  - {M2_S0_ALL_SUMMARY_PATH}")

# 3.16.5 – Logging

log_message(
    "INFO",
    (
        "M2 PR/PCA appliqué à S0 sur TOUTES les fenêtres "
        f"pour {len(df_M2_S0_all)} combinaisons (W,G). "
        f"Résultats sauvegardés dans {M2_S0_ALL_SUMMARY_PATH.name}."
    ),
    block="BLOC_3.16",
)
log_metric(
    "M2_S0_all_windows_combos",
    int(len(df_M2_S0_all)),
    extra={
        "summary_path": str(M2_S0_ALL_SUMMARY_PATH),
    },
)

Nombre total de fenêtres S0 : 11682

M2 PR/PCA sur toutes les fenêtres S0 pour (W=60, G=1) – n_windows=3206
  -> (W=60, G=1) : d_PR=2.740, d_PR_80=2, d_PR_90=7

M2 PR/PCA sur toutes les fenêtres S0 pour (W=60, G=6) – n_windows=535
  -> (W=60, G=6) : d_PR=2.740, d_PR_80=2, d_PR_90=6

M2 PR/PCA sur toutes les fenêtres S0 pour (W=60, G=12) – n_windows=268
  -> (W=60, G=12) : d_PR=2.731, d_PR_80=2, d_PR_90=6

M2 PR/PCA sur toutes les fenêtres S0 pour (W=132, G=1) – n_windows=3134
  -> (W=132, G=1) : d_PR=4.244, d_PR_80=3, d_PR_90=13

M2 PR/PCA sur toutes les fenêtres S0 pour (W=132, G=6) – n_windows=523
  -> (W=132, G=6) : d_PR=4.239, d_PR_80=3, d_PR_90=12

M2 PR/PCA sur toutes les fenêtres S0 pour (W=132, G=12) – n_windows=262
  -> (W=132, G=12) : d_PR=4.225, d_PR_80=3, d_PR_90=11

M2 PR/PCA sur toutes les fenêtres S0 pour (W=264, G=1) – n_windows=3002
  -> (W=264, G=1) : d_PR=5.478, d_PR_80=6, d_PR_90=25

M2 PR/PCA sur toutes les fenêtres S0 pour (W=264, G=6) – n_windows=501
  -> (W=264,

,series,window_size_months,stride_months,n_windows,W,d_PR,d_PR_80,d_PR_90,n_eigvals_pos
0,S0,60,1,3206,60,2.740369,2,7,60
1,S0,60,6,535,60,2.740383,2,6,60
2,S0,60,12,268,60,2.731161,2,6,60
3,S0,132,1,3134,132,4.243610,3,13,132
4,S0,132,6,523,132,4.238866,3,12,132
5,S0,132,12,262,132,4.225176,3,11,132
6,S0,264,1,3002,264,5.478306,6,25,264
7,S0,264,6,501,264,5.468665,6,20,264
8,S0,264,12,251,264,5.447320,6,17,257



Fichier de résultats M2 (S0, PR/PCA, toutes fenêtres) sauvegardé :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates\M2_S0_PR_all_windows_summary.csv
[STEP=24][INFO][BLOC_3.16] M2 PR/PCA appliqué à S0 sur TOUTES les fenêtres pour 9 combinaisons (W,G). Résultats sauvegardés dans M2_S0_PR_all_windows_summary.csv.
[METRIC][M2_S0_all_windows_combos] = 9 (step=24)


### Bloc 3.17 – Méthode M4 (dimension globale Phase 1 comme référence externe)

Pour compléter M1 (Levina–Bickel), M2 (PR/PCA) et M3 (spectrale locale), nous introduisons une
méthode M4 définie comme **référence globale issue de la Phase 1**. En Phase 1, la dimension
spectrale small‑λ robuste de la série Sunspots entière (pipeline `sunspots_external`) donne un
**d_s ≈ 2.0**, avec une incertitude documentée : un bloc interne `d_s_robust_per_b` (médiane ≈ 1.56,
intervalle ≈ [1.38, 1.75]) et un bootstrap externe (médiane ≈ 2.05, IC95% ≈ [1.07, 4.42]).
Dans cette Phase 2, nous utilisons M4 comme **ancre globale** pour S0, indépendante de W et G, afin
de comparer les estimations locales M1/M2/M3 à ce `d_ref` consolidé. Les valeurs de M4 ne sont pas
recalculées ici : elles sont importées de la synthèse Phase 1
([Phase1Sunspots_V0.5_synthese_globale_Tlog.md](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/Phase1Sunspots_V0.5_synthese_globale_Tlog.md:0:0-0:0)) et stockées dans un CSV de référence.

In [27]:
# Bloc 3.17 – Création du fichier de référence M4_global_phase1_reference.csv

import pandas as pd
from pathlib import Path

# 3.17.1 – Dossier pour les estimations de d en Phase 2
D_ESTIMATES_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates"
D_ESTIMATES_DIR.mkdir(parents=True, exist_ok=True)

M4_REF_PATH = D_ESTIMATES_DIR / "M4_global_phase1_reference.csv"

# 3.17.2 – Contenu de la référence M4 (valeurs tirées de la synthèse Phase 1)
data = {
    "method_id": ["M4_global_phase1"],
    "series": ["S0"],
    "d_ref": [2.0],              # d_s ≈ 2.0 (configuration finale sunspots_external)
    "d_internal_median": [1.56], # d_s_robust médiane ≈ 1.56 (ds_robust_per_b)
    "d_internal_ci_low": [1.38], # intervalle interne ≈ [1.38, 1.75]
    "d_internal_ci_high": [1.75],
    "d_external_median": [2.05], # bootstrap externe : médiane ≈ 2.05
    "d_external_ci_low": [1.07], # IC95% ≈ [1.07, 4.42]
    "d_external_ci_high": [4.42],
    "source_summary_file": ["Phase1Sunspots_V0.5_synthese_globale_Tlog.md"],
}

df_m4 = pd.DataFrame(data)

# 3.17.3 – Sauvegarde du CSV de référence
df_m4.to_csv(M4_REF_PATH, index=False)

print("Fichier de référence M4 (Phase 1) écrit à :")
print(f"  - {M4_REF_PATH}")

# 3.17.4 – Logging pour audit
log_message(
    "INFO",
    f"Fichier de référence M4_global_phase1 sauvegardé dans {M4_REF_PATH.name}",
    block="BLOC_3.17",
)
log_metric(
    "M4_global_phase1_reference_written",
    True,
    extra={
        "path": str(M4_REF_PATH),
        "d_ref": 2.0,
        "d_internal_median": 1.56,
        "d_internal_ci": [1.38, 1.75],
        "d_external_median": 2.05,
        "d_external_ci": [1.07, 4.42],
    },
)

Fichier de référence M4 (Phase 1) écrit à :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates\M4_global_phase1_reference.csv
[STEP=25][INFO][BLOC_3.17] Fichier de référence M4_global_phase1 sauvegardé dans M4_global_phase1_reference.csv
[METRIC][M4_global_phase1_reference_written] = True (step=25)


In [28]:
# Bloc 3.18 – Rechargement et affichage de la référence M4_global_phase1

import pandas as pd
from pathlib import Path

# 3.18.1 – Chemin vers le fichier de référence M4
D_ESTIMATES_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates"
M4_REF_PATH = D_ESTIMATES_DIR / "M4_global_phase1_reference.csv"

print(f"Fichier de référence M4 attendu : {M4_REF_PATH}")

if not M4_REF_PATH.exists():
    raise FileNotFoundError(
        f"M4_global_phase1_reference.csv est introuvable à : {M4_REF_PATH}.\n"
        "Assurez-vous d'avoir exécuté le bloc qui crée ce fichier (BLOC_3.17)."
    )

# 3.18.2 – Rechargement et affichage
df_m4_ref = pd.read_csv(M4_REF_PATH)

print("\nContenu de M4_global_phase1_reference.csv :")
display(df_m4_ref)

# 3.18.3 – Logging pour audit (optionnel mais cohérent avec le reste du notebook)
log_message(
    "INFO",
    f"Référence M4_global_phase1 rechargée depuis {M4_REF_PATH.name} "
    f"avec shape={df_m4_ref.shape}",
    block="BLOC_3.18",
)
log_metric(
    "M4_global_phase1_reference_reloaded_rows",
    int(df_m4_ref.shape[0]),
    extra={"n_cols": int(df_m4_ref.shape[1])},
)

Fichier de référence M4 attendu : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates\M4_global_phase1_reference.csv

Contenu de M4_global_phase1_reference.csv :


,method_id,series,d_ref,d_internal_median,d_internal_ci_low,d_internal_ci_high,d_external_median,d_external_ci_low,d_external_ci_high,source_summary_file
0,M4_global_phase1,S0,2.0,1.56,1.38,1.75,2.05,1.07,4.42,Phase1Sunspots_V0.5_synthese_globale_Tlog.md


[STEP=26][INFO][BLOC_3.18] Référence M4_global_phase1 rechargée depuis M4_global_phase1_reference.csv avec shape=(1, 10)
[METRIC][M4_global_phase1_reference_reloaded_rows] = 1 (step=26)


### Bloc 3.19 – Résumé M2 (PR/PCA) par taille de fenêtre W

Nous avons appliqué M2 (PR/PCA) à **toutes les fenêtres S0** pour les 9 combinaisons
(W,G) et sauvegardé les résultats agrégés dans :
`data_phase2/d_estimates/M2_S0_PR_all_windows_summary.csv`.

Dans ce bloc, nous résumons ces résultats par **taille de fenêtre W** en définissant,
pour chaque W ∈ {60, 132, 264}, un `d_PR_ref(W)` comme la **moyenne de d_PR sur les trois strides**
G ∈ {1, 6, 12}. Nous calculons également l’écart‑type de d_PR entre strides, ainsi que des
versions agrégées de `d_PR_80` et `d_PR_90`. Le tableau résultant est sauvegardé dans :

`data_phase2/d_estimates/M2_S0_PR_reference_by_W.csv`

et servira d’entrée principale pour le Bloc 4 (calcul de \( T_{\log}(n_W, d) \)), en parallèle de
la référence globale M4 issue de la Phase 1.

In [29]:
# Bloc 3.19 – Construction et sauvegarde de M2_S0_PR_reference_by_W.csv

import pandas as pd
from pathlib import Path

# 3.19.1 – Chemins des fichiers d'estimation de d (Phase 2)
D_ESTIMATES_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates"
D_ESTIMATES_DIR.mkdir(parents=True, exist_ok=True)

M2_ALL_SUMMARY_PATH = D_ESTIMATES_DIR / "M2_S0_PR_all_windows_summary.csv"
M2_REF_BY_W_PATH = D_ESTIMATES_DIR / "M2_S0_PR_reference_by_W.csv"

print("Fichier M2 (S0, PR/PCA, toutes fenêtres) attendu :")
print(f"  - {M2_ALL_SUMMARY_PATH}")

if not M2_ALL_SUMMARY_PATH.exists():
    raise FileNotFoundError(
        f"M2_S0_PR_all_windows_summary.csv est introuvable à : {M2_ALL_SUMMARY_PATH}.\n"
        "Assurez-vous d'avoir exécuté le bloc qui calcule M2 sur toutes les fenêtres S0."
    )

# 3.19.2 – Chargement du résumé M2_all
df_m2_all = pd.read_csv(M2_ALL_SUMMARY_PATH)

print("\nAperçu de M2_S0_PR_all_windows_summary.csv :")
display(df_m2_all.head())

# Vérification minimale des colonnes attendues
required_cols = [
    "series",
    "window_size_months",
    "stride_months",
    "n_windows",
    "d_PR",
    "d_PR_80",
    "d_PR_90",
]
missing = [c for c in required_cols if c not in df_m2_all.columns]
if missing:
    raise RuntimeError(
        f"Colonnes manquantes dans M2_S0_PR_all_windows_summary.csv : {missing}"
    )

# 3.19.3 – Agrégation par (series, W) pour définir d_PR_ref(W)
group_cols = ["series", "window_size_months"]

df_m2_ref = (
    df_m2_all.groupby(group_cols)
    .agg(
        n_strides=("stride_months", "nunique"),
        strides_list=("stride_months", lambda x: sorted(x.unique())),
        n_windows_total=("n_windows", "sum"),
        d_PR_ref=("d_PR", "mean"),
        d_PR_ref_std=("d_PR", "std"),
        d_PR_80_ref=("d_PR_80", "mean"),
        d_PR_90_ref=("d_PR_90", "mean"),
    )
    .reset_index()
)

# Renommer window_size_months -> W pour plus de lisibilité
df_m2_ref = df_m2_ref.rename(columns={"window_size_months": "W"})

print("\nRésumé M2 S0 par taille de fenêtre W :")
display(df_m2_ref)

# 3.19.4 – Sauvegarde du tableau de référence par W
df_m2_ref.to_csv(M2_REF_BY_W_PATH, index=False)

print("\nFichier de référence M2 (PR/PCA) par W écrit à :")
print(f"  - {M2_REF_BY_W_PATH}")

# 3.19.5 – Logging pour audit
log_message(
    "INFO",
    (
        "Résumé M2 (PR/PCA) pour S0 agrégé par taille de fenêtre W. "
        f"Fichier de référence écrit dans {M2_REF_BY_W_PATH.name}"
    ),
    block="BLOC_3.19",
)
log_metric(
    "M2_S0_PR_reference_by_W_rows",
    int(df_m2_ref.shape[0]),
    extra={
        "n_cols": int(df_m2_ref.shape[1]),
        "W_values": df_m2_ref["W"].tolist(),
        "d_PR_ref": df_m2_ref["d_PR_ref"].tolist(),
    },
)

Fichier M2 (S0, PR/PCA, toutes fenêtres) attendu :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates\M2_S0_PR_all_windows_summary.csv

Aperçu de M2_S0_PR_all_windows_summary.csv :


,series,window_size_months,stride_months,n_windows,W,d_PR,d_PR_80,d_PR_90,n_eigvals_pos
0,S0,60,1,3206,60,2.740369,2,7,60
1,S0,60,6,535,60,2.740383,2,6,60
2,S0,60,12,268,60,2.731161,2,6,60
3,S0,132,1,3134,132,4.243610,3,13,132
4,S0,132,6,523,132,4.238866,3,12,132



Résumé M2 S0 par taille de fenêtre W :


,series,W,n_strides,strides_list,n_windows_total,d_PR_ref,d_PR_ref_std,d_PR_80_ref,d_PR_90_ref
0,S0,60,3,"[1, 6, 12]",4009,2.737305,0.005320,2.0,6.333333
1,S0,132,3,"[1, 6, 12]",3919,4.235884,0.009572,3.0,12.000000
2,S0,264,3,"[1, 6, 12]",3754,5.464764,0.015857,6.0,20.666667



Fichier de référence M2 (PR/PCA) par W écrit à :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates\M2_S0_PR_reference_by_W.csv
[STEP=27][INFO][BLOC_3.19] Résumé M2 (PR/PCA) pour S0 agrégé par taille de fenêtre W. Fichier de référence écrit dans M2_S0_PR_reference_by_W.csv
[METRIC][M2_S0_PR_reference_by_W_rows] = 3 (step=27)


### Bloc 3.20 – Synthèse M2 par W et lien avec M4

Le résumé M2 (PR/PCA) sur **toutes les fenêtres S0** donne, après agrégation par taille de
fenêtre W, les valeurs de référence suivantes (moyenne sur G = 1, 6, 12) :

- W = 60  : `d_PR_ref ≈ 2.74`  (écart-type entre strides ≈ 0.005)
- W = 132 : `d_PR_ref ≈ 4.24`  (écart-type ≈ 0.010)
- W = 264 : `d_PR_ref ≈ 5.46`  (écart-type ≈ 0.016)

La très faible dépendance à G montre que, pour un W donné, M2 est **quasi invariante au stride** :
on peut donc utiliser `d_PR_ref(W)` comme estimateur principal de `d` par taille de fenêtre dans
la Phase 2. Comparée à la référence globale Phase 1 (M4, `d_ref ≈ 2.0`), la structure est cohérente :
pour les petites fenêtres (W = 60), on reste proche de l’ancre globale, tandis que l’augmentation
de W fait croître `d_PR_ref` au-dessus du seuil critique `d = 4`, ce qui sera déterminant pour
l’analyse des régimes de \( T_{\log}(n_W, d) \) dans le Bloc 4.

## Bloc 4 – Calcul de \( T_{\log}(n, d) \)

Dans ce bloc, nous préparons les **entrées structurées** pour le calcul de \( T_{\log}(n, d) \)
à partir des estimations de dimension obtenues en Bloc 3. Nous utiliserons :

- les estimations locales par taille de fenêtre issues de M2 (PR/PCA), résumées dans  
  `data_phase2/d_estimates/M2_S0_PR_reference_by_W.csv` (colonnes `W`, `d_PR_ref`, etc.) ;
- la référence globale M4 issue de la Phase 1, stockée dans  
  `data_phase2/d_estimates/M4_global_phase1_reference.csv` (colonnes `d_ref`, intervalles internes/externes).

Pour chaque taille de fenêtre \( W \in \{60, 132, 264\} \), nous définirons :
- un nombre de points par fenêtre \( n_W = W \) (taille en mois de la fenêtre) ;
- des paires \((n_W, d)\) pour différentes méthodes (`M2_PR_ref`, `M4_global_phase1`).

La cellule de code suivante construit un tableau d’entrée standardisé, fusionnant ces informations
et le sauvegarde dans :

`data_phase2/tlog_inputs/Tlog_inputs_W.csv`

Ce fichier servira de base au calcul et à la comparaison de \( T_{\log}(n_W, d) \) dans les blocs 4.x.

In [30]:
# Bloc 4.0 – Préparation du tableau d'entrée T_log : Tlog_inputs_W.csv

import pandas as pd
from pathlib import Path

# 4.0.1 – Dossiers et chemins des fichiers nécessaires
D_ESTIMATES_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates"
TLOG_INPUTS_DIR = PHASE2_ROOT / "data_phase2" / "tlog_inputs"
TLOG_INPUTS_DIR.mkdir(parents=True, exist_ok=True)

M2_REF_BY_W_PATH = D_ESTIMATES_DIR / "M2_S0_PR_reference_by_W.csv"
M4_REF_PATH = D_ESTIMATES_DIR / "M4_global_phase1_reference.csv"
TLOG_INPUTS_W_PATH = TLOG_INPUTS_DIR / "Tlog_inputs_W.csv"

print("Fichiers d'estimation de d attendus :")
print(f"  - M2_S0_PR_reference_by_W : {M2_REF_BY_W_PATH}")
print(f"  - M4_global_phase1_reference : {M4_REF_PATH}")

if not M2_REF_BY_W_PATH.exists():
    raise FileNotFoundError(
        f"M2_S0_PR_reference_by_W.csv est introuvable à : {M2_REF_BY_W_PATH}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 3.19."
    )

if not M4_REF_PATH.exists():
    raise FileNotFoundError(
        f"M4_global_phase1_reference.csv est introuvable à : {M4_REF_PATH}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 3.17."
    )

# 4.0.2 – Chargement des références M2 (par W) et M4 (globale Phase 1)
df_m2_ref = pd.read_csv(M2_REF_BY_W_PATH)
df_m4_ref = pd.read_csv(M4_REF_PATH)

print("\nRésumé de M2_S0_PR_reference_by_W.csv :")
display(df_m2_ref)

print("\nRésumé de M4_global_phase1_reference.csv :")
display(df_m4_ref)

# Vérifications minimales sur les colonnes attendues
required_m2_cols = ["series", "W", "d_PR_ref", "d_PR_ref_std", "d_PR_80_ref", "d_PR_90_ref"]
missing_m2 = [c for c in required_m2_cols if c not in df_m2_ref.columns]
if missing_m2:
    raise RuntimeError(
        f"Colonnes manquantes dans M2_S0_PR_reference_by_W.csv : {missing_m2}"
    )

required_m4_cols = [
    "method_id",
    "series",
    "d_ref",
    "d_internal_median",
    "d_internal_ci_low",
    "d_internal_ci_high",
    "d_external_median",
    "d_external_ci_low",
    "d_external_ci_high",
    "source_summary_file",
]
missing_m4 = [c for c in required_m4_cols if c not in df_m4_ref.columns]
if missing_m4:
    raise RuntimeError(
        f"Colonnes manquantes dans M4_global_phase1_reference.csv : {missing_m4}"
    )

# 4.0.3 – Construction des lignes pour M2 (PR/PCA) par W
rows = []

for _, row in df_m2_ref.iterrows():
    series = row["series"]
    W = int(row["W"])
    n_points = W  # n_W = W (taille de la fenêtre en points)

    rows.append(
        {
            "series": series,
            "W": W,
            "n_points_per_window": n_points,
            "method_id": "M2_PR_ref",
            "d_est": float(row["d_PR_ref"]),
            "d_est_std": float(row["d_PR_ref_std"]) if pd.notna(row["d_PR_ref_std"]) else None,
            "d_est_80": float(row["d_PR_80_ref"]) if pd.notna(row["d_PR_80_ref"]) else None,
            "d_est_90": float(row["d_PR_90_ref"]) if pd.notna(row["d_PR_90_ref"]) else None,
            "d_ci_internal_low": None,
            "d_ci_internal_high": None,
            "d_ci_external_low": None,
            "d_ci_external_high": None,
            "source_table": "M2_S0_PR_reference_by_W.csv",
        }
    )

# 4.0.4 – Construction des lignes pour M4 (référence globale Phase 1) par W
# M4 est global, mais pour T_log nous voulons pouvoir comparer T_log(n_W, d_ref) pour chaque W.
if len(df_m4_ref) != 1:
    raise RuntimeError(
        "M4_global_phase1_reference.csv devrait contenir exactement 1 ligne."
    )

m4 = df_m4_ref.iloc[0]

for _, row in df_m2_ref.iterrows():
    series = m4["series"]
    W = int(row["W"])
    n_points = W  # même n_W = W

    rows.append(
        {
            "series": series,
            "W": W,
            "n_points_per_window": n_points,
            "method_id": m4["method_id"],
            "d_est": float(m4["d_ref"]),
            "d_est_std": None,
            "d_est_80": None,
            "d_est_90": None,
            "d_ci_internal_low": float(m4["d_internal_ci_low"]),
            "d_ci_internal_high": float(m4["d_internal_ci_high"]),
            "d_ci_external_low": float(m4["d_external_ci_low"]),
            "d_ci_external_high": float(m4["d_external_ci_high"]),
            "source_table": "M4_global_phase1_reference.csv",
        }
    )

# 4.0.5 – DataFrame final d'entrée T_log
df_tlog_inputs = pd.DataFrame(rows)

# Tri pour lisibilité : par W puis par method_id
df_tlog_inputs = df_tlog_inputs.sort_values(["W", "method_id"]).reset_index(drop=True)

print("\nTableau d'entrée T_log (par W et méthode) :")
display(df_tlog_inputs)

# 4.0.6 – Sauvegarde sur disque
df_tlog_inputs.to_csv(TLOG_INPUTS_W_PATH, index=False)

print("\nFichier Tlog_inputs_W écrit à :")
print(f"  - {TLOG_INPUTS_W_PATH}")

# 4.0.7 – Logging pour audit
log_message(
    "INFO",
    (
        "Tableau d'entrée T_log construit à partir de M2_PR_ref(W) et M4_global_phase1. "
        f"Fichier sauvegardé dans {TLOG_INPUTS_W_PATH.name}"
    ),
    block="BLOC_4.0",
)
log_metric(
    "Tlog_inputs_W_rows",
    int(df_tlog_inputs.shape[0]),
    extra={
        "n_cols": int(df_tlog_inputs.shape[1]),
        "W_values": sorted(df_tlog_inputs["W"].unique().tolist()),
        "methods": sorted(df_tlog_inputs["method_id"].unique().tolist()),
    },
)

Fichiers d'estimation de d attendus :
  - M2_S0_PR_reference_by_W : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates\M2_S0_PR_reference_by_W.csv
  - M4_global_phase1_reference : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates\M4_global_phase1_reference.csv



Résumé de M2_S0_PR_reference_by_W.csv :


,series,W,n_strides,strides_list,n_windows_total,d_PR_ref,d_PR_ref_std,d_PR_80_ref,d_PR_90_ref
0,S0,60,3,"[np.int64(1), np.int64(6), np.int64(12)]",4009,2.737305,0.005320,2.0,6.333333
1,S0,132,3,"[np.int64(1), np.int64(6), np.int64(12)]",3919,4.235884,0.009572,3.0,12.000000
2,S0,264,3,"[np.int64(1), np.int64(6), np.int64(12)]",3754,5.464764,0.015857,6.0,20.666667



Résumé de M4_global_phase1_reference.csv :


,method_id,series,d_ref,d_internal_median,d_internal_ci_low,d_internal_ci_high,d_external_median,d_external_ci_low,d_external_ci_high,source_summary_file
0,M4_global_phase1,S0,2.0,1.56,1.38,1.75,2.05,1.07,4.42,Phase1Sunspots_V0.5_synthese_globale_Tlog.md



Tableau d'entrée T_log (par W et méthode) :


,series,W,n_points_per_window,method_id,d_est,d_est_std,d_est_80,d_est_90,d_ci_internal_low,d_ci_internal_high,d_ci_external_low,d_ci_external_high,source_table
0,S0,60,60,M2_PR_ref,2.737305,0.005320,2.0,6.333333,NaN,NaN,NaN,NaN,M2_S0_PR_reference_by_W.csv
1,S0,60,60,M4_global_phase1,2.000000,NaN,NaN,NaN,1.38,1.75,1.07,4.42,M4_global_phase1_reference.csv
2,S0,132,132,M2_PR_ref,4.235884,0.009572,3.0,12.000000,NaN,NaN,NaN,NaN,M2_S0_PR_reference_by_W.csv
3,S0,132,132,M4_global_phase1,2.000000,NaN,NaN,NaN,1.38,1.75,1.07,4.42,M4_global_phase1_reference.csv
4,S0,264,264,M2_PR_ref,5.464764,0.015857,6.0,20.666667,NaN,NaN,NaN,NaN,M2_S0_PR_reference_by_W.csv
5,S0,264,264,M4_global_phase1,2.000000,NaN,NaN,NaN,1.38,1.75,1.07,4.42,M4_global_phase1_reference.csv



Fichier Tlog_inputs_W écrit à :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\tlog_inputs\Tlog_inputs_W.csv
[STEP=28][INFO][BLOC_4.0] Tableau d'entrée T_log construit à partir de M2_PR_ref(W) et M4_global_phase1. Fichier sauvegardé dans Tlog_inputs_W.csv
[METRIC][Tlog_inputs_W_rows] = 6 (step=28)


### Bloc 4.1 – Calcul de \( T_{\log}(n_W, d) \) pour M2 et M4

À partir du tableau d'entrée `Tlog_inputs_W.csv`, nous calculons maintenant
\( T_{\log}(n_W, d) \) pour chaque combinaison (W, méthode) en utilisant la
même convention que la Phase 1 :

\[
T_{\log}(n, d) = (d - 4)\,\ln(n), \quad \text{(bias = 0)}.
\]

Pour chaque taille de fenêtre \( W \), nous avons défini \( n_W = W \) comme nombre de
points par fenêtre. Nous calculerons :

- une valeur centrale `T_log` pour chaque méthode (M2\_PR\_ref, M4\_global_phase1) ;
- lorsque disponible, des variantes ou intervalles dérivés des bornes sur `d` :
  - `T_log_80`, `T_log_90` à partir de `d_est_80`, `d_est_90` (M2) ;
  - `T_log_ci_internal_*`, `T_log_ci_external_*` à partir des intervalles de M4.

Les résultats seront sauvegardés dans un fichier dédié :

`data_phase2/tlog_results/Tlog_values_W.csv`

qui servira de base aux blocs suivants (4.x) pour analyser les régimes
(Divergence / Équilibre / Saturation) en fonction de W et de la méthode d’estimation de `d`.